#SET UP

In [ ]:
# install java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://downloads.apache.org/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz

# unzip the spark file to the current folder
!tar xf spark-3.5.1-bin-hadoop3.tgz

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 229 kB in 2s (108 kB/s)
Reading package lists... Done


In [ ]:
# set your spark folder to your system path environment.
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

In [ ]:
!pip install findspark
import findspark
findspark.init()
import pyspark

In [ ]:
# Creating a SparkSession in Python
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local")\
          .appName("Spark Streaming Demonstration")\
          .config("spark.some.config.option", "some-value")\
          .getOrCreate()

# keep the size of shuffles small
spark.conf.set("spark.sql.shuffle.partitions", "2")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# input path in gDrive
inputPath = '/content/drive/MyDrive/Lab 3/taxi-data'

In [ ]:
# Check if we can access the data files
import os
os.listdir(inputPath)

['part-2015-12-01-1206.csv',
 'part-2015-12-01-0435.csv',
 'part-2015-12-01-0822.csv',
 'part-2015-12-01-1549.csv',
 'part-2015-12-01-1550.csv',
 'part-2015-12-01-0320.csv',
 'part-2015-12-01-1329.csv',
 'part-2015-12-01-0205.csv',
 'part-2015-12-01-1619.csv',
 'part-2015-12-01-0017.csv',
 'part-2015-12-01-0642.csv',
 'part-2015-12-01-0827.csv',
 'part-2015-12-01-2341.csv',
 'part-2015-12-01-0855.csv',
 'part-2015-12-01-0747.csv',
 'part-2015-12-01-0136.csv',
 'part-2015-12-01-0627.csv',
 'part-2015-12-01-1631.csv',
 'part-2015-12-01-0028.csv',
 'part-2015-12-01-0801.csv',
 'part-2015-12-01-0547.csv',
 'part-2015-12-01-0030.csv',
 'part-2015-12-01-0615.csv',
 'part-2015-12-01-1400.csv',
 'part-2015-12-01-0340.csv',
 'part-2015-12-01-1023.csv',
 'part-2015-12-01-2322.csv',
 'part-2015-12-01-0116.csv',
 'part-2015-12-01-1640.csv',
 'part-2015-12-01-1117.csv',
 'part-2015-12-01-1223.csv',
 'part-2015-12-01-1606.csv',
 'part-2015-12-01-1025.csv',
 'part-2015-12-01-0423.csv',
 'part-2015-12

#STREAMING


* Yellow Taxis pick up street-hailing passengers anywhere in NYC.
* Green Taxis pick up street-hailing passengers in NYC **except for** below East 96th Street and West 110th Street in Manhattan.


##Task 1

In [ ]:
from pyspark.sql.types import *

temp_schema = StructType(
    [
        StructField("_c1", StringType(), True), # type
        StructField("_c2", StringType(), True), # VendorID
        StructField("_c3", StringType(), True), # pickup_datetime
        StructField("_c4", StringType(), True), # dropoff_datetime
        StructField("_c5", StringType(), True),
        StructField("_c6", StringType(), True),
        StructField("_c7", StringType(), True),
        StructField("_c8", StringType(), True),
        StructField("_c9", StringType(), True),
        StructField("_c10", StringType(), True),
        StructField("_c11", StringType(), True),
        StructField("_c12", StringType(), True),
        StructField("_c13", StringType(), True),
        StructField("_c14", StringType(), True),
        StructField("_c15", StringType(), True),
        StructField("_c16", StringType(), True),
        StructField("_c17", StringType(), True),
        StructField("_c18", StringType(), True),
        StructField("_c19", StringType(), True),
        StructField("_c20", StringType(), True),
        StructField("_c21", StringType(), True),
        StructField("_c22", StringType(), True),
    ]
)

In [ ]:
from pyspark.sql.functions import *

# Similar to definition of staticInputDF above, just using `readStream` instead of `read`
streamingInputDF = (
    spark
        .readStream
        .schema(temp_schema)                # Set the schema of the csv data
        .option("maxFilesPerTrigger", 8)    # Treat a sequence of files as a stream by picking one file at a time
        .csv(inputPath)
)

In [ ]:
# This query stores the results in memory then visualize it
query = (
    streamingInputDF
        .writeStream
        .format("memory")         # console or memory(= store in-memory table)
        .queryName("input")       # counts = name of the in-memory table
        .outputMode("append")
        .option("truncate", "false")
        .start()
)
query.awaitTermination(30)

spark.sql('select (*) from input').show(truncate=False)

+------+---+-------------------+-------------------+---+-----+-------------------+------------------+-------------------+------------------+-------------------+------------------+----+----+----+----+-----+----+----+-----+----+----+
|_c1   |_c2|_c3                |_c4                |_c5|_c6  |_c7                |_c8               |_c9                |_c10              |_c11               |_c12              |_c13|_c14|_c15|_c16|_c17 |_c18|_c19|_c20 |_c21|_c22|
+------+---+-------------------+-------------------+---+-----+-------------------+------------------+-------------------+------------------+-------------------+------------------+----+----+----+----+-----+----+----+-----+----+----+
|yellow|2  |2015-12-01 08:07:20|2015-12-01 08:22:00|2  |1.60 |-74.00714111328125 |40.718009948730469|1                  |N                 |-73.990066528320312|40.724758148193359|2   |11  |0   |0.5 |0    |0   |0.3 |11.8 |NULL|NULL|
|yellow|2  |2015-12-01 08:07:27|2015-12-01 08:22:00|1  |1.68 |-73.967636

##Task 2

In [ ]:
# cast the Timestamp type
dropoff_datetime_DF = (
    streamingInputDF
        .select(streamingInputDF['_c4']
        .cast("timestamp")
        .alias("dropoff_datetime"))
)

# groupBy count
count_dropoff_datetime_DF = (
    dropoff_datetime_DF
        .withWatermark("dropoff_datetime", "5 minutes")
        .groupBy(window("dropoff_datetime", "1 hour"))
        .count()
        .withColumn("timestamp",
                    when(hour(col("window")["end"]) == 0, 24 * 360_000)
                    .otherwise(hour(col("window")["end"]) * 360_000))
)

# Is this DF actually a streaming DF?
count_dropoff_datetime_DF.isStreaming

True

In [ ]:
# This query stores the aggregation results in memory then visualize it
EventCount = (
    count_dropoff_datetime_DF
        .writeStream
        .format("memory")              # console or memory(= store in-memory table)
        .queryName("event_count")      # counts = name of the in-memory table
        .outputMode("complete")
        .option("truncate", "false")
        .start()
)
EventCount.awaitTermination(300)

spark.sql('select (*) from event_count order by window').show(truncate=False)

+------------------------------------------+-----+---------+
|window                                    |count|timestamp|
+------------------------------------------+-----+---------+
|{2015-12-01 00:00:00, 2015-12-01 01:00:00}|7396 |360000   |
|{2015-12-01 01:00:00, 2015-12-01 02:00:00}|5780 |720000   |
|{2015-12-01 02:00:00, 2015-12-01 03:00:00}|3605 |1080000  |
|{2015-12-01 03:00:00, 2015-12-01 04:00:00}|2426 |1440000  |
|{2015-12-01 04:00:00, 2015-12-01 05:00:00}|2505 |1800000  |
|{2015-12-01 05:00:00, 2015-12-01 06:00:00}|3858 |2160000  |
|{2015-12-01 06:00:00, 2015-12-01 07:00:00}|10258|2520000  |
|{2015-12-01 07:00:00, 2015-12-01 08:00:00}|19007|2880000  |
|{2015-12-01 08:00:00, 2015-12-01 09:00:00}|23799|3240000  |
|{2015-12-01 09:00:00, 2015-12-01 10:00:00}|24003|3600000  |
|{2015-12-01 10:00:00, 2015-12-01 11:00:00}|21179|3960000  |
|{2015-12-01 11:00:00, 2015-12-01 12:00:00}|20219|4320000  |
|{2015-12-01 12:00:00, 2015-12-01 13:00:00}|20522|4680000  |
|{2015-12-01 13:00:00, 2

In [ ]:
import os

output_path = '/content/drive/MyDrive/Lab 3/task2-output'
checkpoint_path = '/content/drive/MyDrive/Lab 3/task2-checkpoint'

if not os.path.exists(output_path):
    os.makedirs(output_path)

if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)

In [ ]:
# Start the query to write each window into a separate directory
query = (
    count_dropoff_datetime_DF
        .drop("window")
        .withColumnRenamed("timestamp", "output")
        .writeStream
        .format("csv")
        .partitionBy("output")  # Partition by the timestamp
        .option("path", output_path)  # /task2-output
        .option("checkpointLocation", checkpoint_path) \
        .outputMode("append")
        .start()
)

query.awaitTermination(900)

False

In [ ]:
task2_result_df = spark.sql('select (*) from event_count order by window')
task2_result_df.show(truncate=False)

+------------------------------------------+-----+---------+
|window                                    |count|timestamp|
+------------------------------------------+-----+---------+
|{2015-12-01 00:00:00, 2015-12-01 01:00:00}|7396 |360000   |
|{2015-12-01 01:00:00, 2015-12-01 02:00:00}|5780 |720000   |
|{2015-12-01 02:00:00, 2015-12-01 03:00:00}|3605 |1080000  |
|{2015-12-01 03:00:00, 2015-12-01 04:00:00}|2426 |1440000  |
|{2015-12-01 04:00:00, 2015-12-01 05:00:00}|2505 |1800000  |
|{2015-12-01 05:00:00, 2015-12-01 06:00:00}|3858 |2160000  |
|{2015-12-01 06:00:00, 2015-12-01 07:00:00}|10258|2520000  |
|{2015-12-01 07:00:00, 2015-12-01 08:00:00}|19007|2880000  |
|{2015-12-01 08:00:00, 2015-12-01 09:00:00}|23799|3240000  |
|{2015-12-01 09:00:00, 2015-12-01 10:00:00}|24003|3600000  |
|{2015-12-01 10:00:00, 2015-12-01 11:00:00}|21179|3960000  |
|{2015-12-01 11:00:00, 2015-12-01 12:00:00}|20219|4320000  |
|{2015-12-01 12:00:00, 2015-12-01 13:00:00}|20522|4680000  |
|{2015-12-01 13:00:00, 2

In [ ]:
import os

output_path = '/content/drive/MyDrive/Lab 3/task2-output-backup'

if not os.path.exists(output_path):
    os.makedirs(output_path)

In [ ]:
# Iterate over the rows of the DataFrame and write each row to a separate file
for row in task2_result_df.collect():
    count = row['count']
    start_time = row['window']['start'].strftime("%Y-%m-%d %H:%M:%S")
    end_time = row['window']['end'].strftime("%Y-%m-%d %H:%M:%S")

    hour_value = int(row['window']['end'].strftime("%H"))  # Extract the hour from the end time

    if hour_value == 0:
        hour_value = 24
    file_name = f"output-{hour_value * 360_000}.txt"

    with open(os.path.join(output_path, file_name), 'w') as file:
        row_str = f"Window Start: {start_time}\nWindow End: {end_time}\nCount: {count}"
        file.write(row_str)

##Task 3

In [ ]:
# YELLOW: C11, C12
# GREEN: C9, C10

dropoff_DF = streamingInputDF.select(
    when(col('_c1') == 'yellow', col('_c11')).otherwise(col('_c9')).cast("double").alias('dropoff_longitude'),
    when(col('_c1') == 'yellow', col('_c12')).otherwise(col('_c10')).cast("double").alias('dropoff_latitude'),
    col('_c4').cast("timestamp").alias("dropoff_datetime")
)

# Is this DF actually a streaming DF?
dropoff_DF.isStreaming

True

In [ ]:
# This query stores the aggregation results in memory then visualize it
dropoff_query = (
    dropoff_DF
        .writeStream
        .format("memory")         # console or memory(= store in-memory table)
        .queryName("dropoff_input")      # counts = name of the in-memory table
        .outputMode("append")
        .option("truncate", "false")
        .start()
)
dropoff_query.awaitTermination(60)

spark.sql('select (*) from dropoff_input').show(truncate=False)

+------------------+------------------+-------------------+
|dropoff_longitude |dropoff_latitude  |dropoff_datetime   |
+------------------+------------------+-------------------+
|-73.99006652832031|40.72475814819336 |2015-12-01 08:22:00|
|-73.97795104980469|40.7486686706543  |2015-12-01 08:22:00|
|-73.97513580322266|40.75277328491211 |2015-12-01 08:22:00|
|-73.9623794555664 |40.767189025878906|2015-12-01 08:22:00|
|-73.9886474609375 |40.753150939941406|2015-12-01 08:22:00|
|-73.98548889160156|40.74140167236328 |2015-12-01 08:22:00|
|-73.88993835449219|40.846805572509766|2015-12-01 08:22:00|
|-73.97801971435547|40.75691604614258 |2015-12-01 08:22:00|
|-73.93547821044922|40.85005187988281 |2015-12-01 08:22:00|
|-73.98455810546875|40.68360900878906 |2015-12-01 08:22:00|
|-73.95647430419922|40.7789306640625  |2015-12-01 08:22:01|
|-73.97428131103516|40.75625991821289 |2015-12-01 08:22:01|
|-73.79430389404297|40.64460754394531 |2015-12-01 08:22:01|
|-73.9871597290039 |40.752803802490234|2

In [ ]:
# Define the bounding boxes
goldman = [[-74.0141012, 40.7152191],
           [-74.013777, 40.7152275],
           [-74.0141027, 40.7138745],
           [-74.0144185, 40.7140753]]

citigroup = [[-74.011869, 40.7217236],
             [-74.009867, 40.721493],
             [-74.010140, 40.720053],
             [-74.012083, 40.720267]]

# Define the point_in_polygon function as a Spark UDF
@udf(returnType=StringType())
def point_in_polygon(point):
    x, y = point
    in_goldman = False
    in_citigroup = False

    # Check if the point is in the Goldman box
    num_crossings = 0
    for i in range(len(goldman)):
        x1, y1 = goldman[i]
        x2, y2 = goldman[(i + 1) % len(goldman)]
        if ((y1 > y) != (y2 > y)) and (x < (x2 - x1) * (y - y1) / (y2 - y1) + x1):
            num_crossings += 1
    in_goldman = num_crossings % 2 == 1

    # Check if the point is in the Citigroup box
    num_crossings = 0
    for i in range(len(citigroup)):
        x1, y1 = citigroup[i]
        x2, y2 = citigroup[(i + 1) % len(citigroup)]
        if ((y1 > y) != (y2 > y)) and (x < (x2 - x1) * (y - y1) / (y2 - y1) + x1):
            num_crossings += 1
    in_citigroup = num_crossings % 2 == 1

    # Determine the state
    if in_goldman:
        return "goldman"
    elif in_citigroup:
        return "citigroup"
    else:
        return "none"

In [ ]:
dropoff_state_DF = (
    dropoff_DF
        .withColumn("state", point_in_polygon(struct("dropoff_longitude", "dropoff_latitude")))
)

# Is this DF actually a streaming DF?
dropoff_state_DF.isStreaming

True

In [ ]:
count_dropoff_at_headquarters_DF = (
    dropoff_state_DF
        .filter((dropoff_state_DF.state == 'goldman') | (dropoff_state_DF.state == 'citigroup'))
        .withWatermark("dropoff_datetime", "5 minutes")
        .groupBy(window("dropoff_datetime", "1 hour"), "state")
        .count()
        .withColumn("timestamp",
                    when(hour(col("window")["end"]) == 0, 24 * 360_000)
                    .otherwise(hour(col("window")["end"]) * 360_000))
)

# Is this DF actually a streaming DF?
count_dropoff_at_headquarters_DF.isStreaming

True

In [ ]:
# This query stores the aggregation results in memory then visualize it
RegionEventCount = (
    count_dropoff_at_headquarters_DF
        .writeStream
        .format("memory")         # console or memory(= store in-memory table)
        .queryName("region_event_count")      # counts = name of the in-memory table
        .outputMode("complete")
        .option("truncate", "false")
        .start()
)
RegionEventCount.awaitTermination(900)

spark.sql('select (*) from region_event_count order by window, state').show(truncate=False)

+------------------------------------------+---------+-----+---------+
|window                                    |state    |count|timestamp|
+------------------------------------------+---------+-----+---------+
|{2015-12-01 00:00:00, 2015-12-01 01:00:00}|citigroup|5    |360000   |
|{2015-12-01 01:00:00, 2015-12-01 02:00:00}|citigroup|2    |720000   |
|{2015-12-01 02:00:00, 2015-12-01 03:00:00}|citigroup|1    |1080000  |
|{2015-12-01 04:00:00, 2015-12-01 05:00:00}|citigroup|1    |1800000  |
|{2015-12-01 05:00:00, 2015-12-01 06:00:00}|citigroup|8    |2160000  |
|{2015-12-01 05:00:00, 2015-12-01 06:00:00}|goldman  |3    |2160000  |
|{2015-12-01 06:00:00, 2015-12-01 07:00:00}|citigroup|46   |2520000  |
|{2015-12-01 06:00:00, 2015-12-01 07:00:00}|goldman  |11   |2520000  |
|{2015-12-01 07:00:00, 2015-12-01 08:00:00}|citigroup|62   |2880000  |
|{2015-12-01 07:00:00, 2015-12-01 08:00:00}|goldman  |17   |2880000  |
|{2015-12-01 08:00:00, 2015-12-01 09:00:00}|citigroup|56   |3240000  |
|{2015

In [ ]:
task3_result_df = spark.sql('select (*) from region_event_count order by window, state')
task3_result_df.show(truncate=False)

+------------------------------------------+---------+-----+---------+
|window                                    |state    |count|timestamp|
+------------------------------------------+---------+-----+---------+
|{2015-12-01 00:00:00, 2015-12-01 01:00:00}|citigroup|5    |360000   |
|{2015-12-01 01:00:00, 2015-12-01 02:00:00}|citigroup|2    |720000   |
|{2015-12-01 02:00:00, 2015-12-01 03:00:00}|citigroup|1    |1080000  |
|{2015-12-01 04:00:00, 2015-12-01 05:00:00}|citigroup|1    |1800000  |
|{2015-12-01 05:00:00, 2015-12-01 06:00:00}|citigroup|8    |2160000  |
|{2015-12-01 05:00:00, 2015-12-01 06:00:00}|goldman  |3    |2160000  |
|{2015-12-01 06:00:00, 2015-12-01 07:00:00}|citigroup|46   |2520000  |
|{2015-12-01 06:00:00, 2015-12-01 07:00:00}|goldman  |11   |2520000  |
|{2015-12-01 07:00:00, 2015-12-01 08:00:00}|citigroup|62   |2880000  |
|{2015-12-01 07:00:00, 2015-12-01 08:00:00}|goldman  |17   |2880000  |
|{2015-12-01 08:00:00, 2015-12-01 09:00:00}|citigroup|56   |3240000  |
|{2015

In [ ]:
import os

output_path = '/content/drive/MyDrive/Lab 3/task3-output-backup'

if not os.path.exists(output_path):
    os.makedirs(output_path)

In [ ]:
# Iterate over the rows of the DataFrame and write each row to a separate file
for row in task3_result_df.collect():
    headquarter = row['state']
    count = row['count']
    start_time = row['window']['start'].strftime("%Y-%m-%d %H:%M:%S")
    end_time = row['window']['end'].strftime("%Y-%m-%d %H:%M:%S")

    hour_value = int(row['window']['end'].strftime("%H"))  # Extract the hour from the end time

    if hour_value == 0:
        hour_value = 24
    file_name = f"output-{hour_value * 360_000}.txt"

    with open(os.path.join(output_path, file_name), 'a') as file:
        row_str = f"\nWindow Start: {start_time}\nWindow End: {end_time}\nHeadquarter: {headquarter}\nCount: {count}"
        file.write(row_str)

##Task 4

##goldman sach

In [ ]:
count_dropoff_at_goldman_DF = (
    dropoff_state_DF
        .filter(dropoff_state_DF.state == 'goldman')
        .withWatermark("dropoff_datetime", "5 minutes")
        .groupBy("state", window("dropoff_datetime", "10 minutes"))
        .count()
)

# Is this DF actually a streaming DF?
count_dropoff_at_goldman_DF.isStreaming

True

In [ ]:
# This query stores the aggregation results in memory then visualize it
goldmanEventCount10m = (
    count_dropoff_at_goldman_DF
        .writeStream
        .format("memory")         # console or memory(= store in-memory table)
        .queryName("goldman_event_count_each_10m")      # counts = name of the in-memory table
        .outputMode("complete")
        .option("truncate", "false")
        .start()
)
goldmanEventCount10m.awaitTermination(1200)

spark.sql('select (*) from goldman_event_count_each_10m order by window').show(truncate=False)

+-------+------------------------------------------+-----+
|state  |window                                    |count|
+-------+------------------------------------------+-----+
|goldman|{2015-12-01 05:20:00, 2015-12-01 05:30:00}|1    |
|goldman|{2015-12-01 05:50:00, 2015-12-01 06:00:00}|2    |
|goldman|{2015-12-01 06:00:00, 2015-12-01 06:10:00}|1    |
|goldman|{2015-12-01 06:10:00, 2015-12-01 06:20:00}|1    |
|goldman|{2015-12-01 06:20:00, 2015-12-01 06:30:00}|1    |
|goldman|{2015-12-01 06:30:00, 2015-12-01 06:40:00}|2    |
|goldman|{2015-12-01 06:40:00, 2015-12-01 06:50:00}|3    |
|goldman|{2015-12-01 06:50:00, 2015-12-01 07:00:00}|3    |
|goldman|{2015-12-01 07:00:00, 2015-12-01 07:10:00}|3    |
|goldman|{2015-12-01 07:20:00, 2015-12-01 07:30:00}|3    |
|goldman|{2015-12-01 07:30:00, 2015-12-01 07:40:00}|5    |
|goldman|{2015-12-01 07:40:00, 2015-12-01 07:50:00}|4    |
|goldman|{2015-12-01 07:50:00, 2015-12-01 08:00:00}|2    |
|goldman|{2015-12-01 08:00:00, 2015-12-01 08:10:00}|6   

In [ ]:
from pyspark.sql.window import Window

# Define a window specification
windowSpec = Window.orderBy("window")

goldmanEventCount10mDF = spark.sql('select (*) from goldman_event_count_each_10m order by window')

goldmanEventCount10mDF_with_prev = (
    goldmanEventCount10mDF
        .withColumn("prev_window", lag("window", offset=1, default=None).over(windowSpec))
        .withColumn("prev_count", lag("count", offset=1, default=None).over(windowSpec))
)

goldmanEventCount10mDF_with_prev.show(truncate=False)

+-------+------------------------------------------+-----+------------------------------------------+----------+
|state  |window                                    |count|prev_window                               |prev_count|
+-------+------------------------------------------+-----+------------------------------------------+----------+
|goldman|{2015-12-01 05:20:00, 2015-12-01 05:30:00}|1    |NULL                                      |NULL      |
|goldman|{2015-12-01 05:50:00, 2015-12-01 06:00:00}|2    |{2015-12-01 05:20:00, 2015-12-01 05:30:00}|1         |
|goldman|{2015-12-01 06:00:00, 2015-12-01 06:10:00}|1    |{2015-12-01 05:50:00, 2015-12-01 06:00:00}|2         |
|goldman|{2015-12-01 06:10:00, 2015-12-01 06:20:00}|1    |{2015-12-01 06:00:00, 2015-12-01 06:10:00}|1         |
|goldman|{2015-12-01 06:20:00, 2015-12-01 06:30:00}|1    |{2015-12-01 06:10:00, 2015-12-01 06:20:00}|1         |
|goldman|{2015-12-01 06:30:00, 2015-12-01 06:40:00}|2    |{2015-12-01 06:20:00, 2015-12-01 06:30

In [ ]:
goldmanEventCountEach10mDF_check_doubled = (
    goldmanEventCount10mDF_with_prev
        .withColumn("has_doubled",
                    when(
                        (col("count") >= 10) &
                        (col("count") >= 2 * col("prev_count")) &
                        (col("window.start") == col("prev_window.end")),
                        "yes")
                    .otherwise("no"))
)

goldmanEventCountEach10mDF_check_doubled.show(truncate=False)

+-------+------------------------------------------+-----+------------------------------------------+----------+-----------+
|state  |window                                    |count|prev_window                               |prev_count|has_doubled|
+-------+------------------------------------------+-----+------------------------------------------+----------+-----------+
|goldman|{2015-12-01 05:20:00, 2015-12-01 05:30:00}|1    |NULL                                      |NULL      |no         |
|goldman|{2015-12-01 05:50:00, 2015-12-01 06:00:00}|2    |{2015-12-01 05:20:00, 2015-12-01 05:30:00}|1         |no         |
|goldman|{2015-12-01 06:00:00, 2015-12-01 06:10:00}|1    |{2015-12-01 05:50:00, 2015-12-01 06:00:00}|2         |no         |
|goldman|{2015-12-01 06:10:00, 2015-12-01 06:20:00}|1    |{2015-12-01 06:00:00, 2015-12-01 06:10:00}|1         |no         |
|goldman|{2015-12-01 06:20:00, 2015-12-01 06:30:00}|1    |{2015-12-01 06:10:00, 2015-12-01 06:20:00}|1         |no         |


In [ ]:
goldmanEventCountEach10mDF_doubled = goldmanEventCountEach10mDF_check_doubled.filter(col("has_doubled") == "yes")
goldmanEventCountEach10mDF_doubled.show(truncate=False)

+-----+------+-----+-----------+----------+-----------+
|state|window|count|prev_window|prev_count|has_doubled|
+-----+------+-----+-----------+----------+-----------+
+-----+------+-----+-----------+----------+-----------+



In [ ]:
# Define a function to format and print the results
def print_goldman_doubled_count(rows):
    for row in rows:
        previous_count = row['prev_count']
        current_count = row['count']
        start_time = row['window']['start'].strftime("%Y-%m-%d %H:%M:%S")
        end_time = row['window']['end'].strftime("%Y-%m-%d %H:%M:%S")
        print(f"The number of arrivals to Goldman Sachs has doubled from {previous_count} to {current_count} at {start_time} - {end_time}")

# Print the results in the specified format
goldman_doubled_rows = goldmanEventCountEach10mDF_doubled.select("prev_count", "count", "window").collect()  # Collect the rows to the driver
print_goldman_doubled_count(goldman_doubled_rows)

##citigroup

In [ ]:
count_dropoff_at_citigroup_DF = (
    dropoff_state_DF
        .filter(dropoff_state_DF.state == 'citigroup')
        .withWatermark("dropoff_datetime", "5 minutes")
        .groupBy("state", window("dropoff_datetime", "10 minutes"))
        .count()
)

# Is this DF actually a streaming DF?
count_dropoff_at_citigroup_DF.isStreaming

True

In [ ]:
# This query stores the aggregation results in memory then visualize it
citigroupEventCount10m = (
    count_dropoff_at_citigroup_DF
        .writeStream
        .format("memory")         # console or memory(= store in-memory table)
        .queryName("citigroup_event_count_each_10m")      # counts = name of the in-memory table
        .outputMode("complete")
        .option("truncate", "false")
        .start()
)
citigroupEventCount10m.awaitTermination(1200)

spark.sql('select (*) from citigroup_event_count_each_10m order by window').show(truncate=False)

+---------+------------------------------------------+-----+
|state    |window                                    |count|
+---------+------------------------------------------+-----+
|citigroup|{2015-12-01 00:10:00, 2015-12-01 00:20:00}|1    |
|citigroup|{2015-12-01 00:20:00, 2015-12-01 00:30:00}|2    |
|citigroup|{2015-12-01 00:50:00, 2015-12-01 01:00:00}|2    |
|citigroup|{2015-12-01 01:00:00, 2015-12-01 01:10:00}|1    |
|citigroup|{2015-12-01 01:40:00, 2015-12-01 01:50:00}|1    |
|citigroup|{2015-12-01 02:40:00, 2015-12-01 02:50:00}|1    |
|citigroup|{2015-12-01 04:00:00, 2015-12-01 04:10:00}|1    |
|citigroup|{2015-12-01 05:10:00, 2015-12-01 05:20:00}|1    |
|citigroup|{2015-12-01 05:40:00, 2015-12-01 05:50:00}|1    |
|citigroup|{2015-12-01 05:50:00, 2015-12-01 06:00:00}|6    |
|citigroup|{2015-12-01 06:10:00, 2015-12-01 06:20:00}|1    |
|citigroup|{2015-12-01 06:20:00, 2015-12-01 06:30:00}|7    |
|citigroup|{2015-12-01 06:30:00, 2015-12-01 06:40:00}|7    |
|citigroup|{2015-12-01 0

In [ ]:
from pyspark.sql.window import Window

# Define a window specification
windowSpec  = Window.orderBy("window")

citigroupEventCount10mDF = spark.sql('select (*) from citigroup_event_count_each_10m order by window')

citigroupEventCount10mDF_with_prev = (
    citigroupEventCount10mDF
        .withColumn("prev_window", lag("window", offset=1, default=None).over(windowSpec))
        .withColumn("prev_count", lag("count", offset=1, default=None).over(windowSpec))
)

citigroupEventCount10mDF_with_prev.show(truncate=False)

+---------+------------------------------------------+-----+------------------------------------------+----------+
|state    |window                                    |count|prev_window                               |prev_count|
+---------+------------------------------------------+-----+------------------------------------------+----------+
|citigroup|{2015-12-01 00:10:00, 2015-12-01 00:20:00}|1    |NULL                                      |NULL      |
|citigroup|{2015-12-01 00:20:00, 2015-12-01 00:30:00}|2    |{2015-12-01 00:10:00, 2015-12-01 00:20:00}|1         |
|citigroup|{2015-12-01 00:50:00, 2015-12-01 01:00:00}|2    |{2015-12-01 00:20:00, 2015-12-01 00:30:00}|2         |
|citigroup|{2015-12-01 01:00:00, 2015-12-01 01:10:00}|1    |{2015-12-01 00:50:00, 2015-12-01 01:00:00}|2         |
|citigroup|{2015-12-01 01:40:00, 2015-12-01 01:50:00}|1    |{2015-12-01 01:00:00, 2015-12-01 01:10:00}|1         |
|citigroup|{2015-12-01 02:40:00, 2015-12-01 02:50:00}|1    |{2015-12-01 01:40:00

In [ ]:
citigroupEventCountEach10mDF_check_doubled = (
    citigroupEventCount10mDF_with_prev
        .withColumn(
            "has_doubled",
            when(
                (col("count") >= 10) &
                (col("count") >= 2 * col("prev_count")) &
                (col("window.start") == col("prev_window.end")),
                "yes")
            .otherwise("no"))
)

citigroupEventCountEach10mDF_check_doubled.show(truncate=False)

+---------+------------------------------------------+-----+------------------------------------------+----------+-----------+
|state    |window                                    |count|prev_window                               |prev_count|has_doubled|
+---------+------------------------------------------+-----+------------------------------------------+----------+-----------+
|citigroup|{2015-12-01 00:10:00, 2015-12-01 00:20:00}|1    |NULL                                      |NULL      |no         |
|citigroup|{2015-12-01 00:20:00, 2015-12-01 00:30:00}|2    |{2015-12-01 00:10:00, 2015-12-01 00:20:00}|1         |no         |
|citigroup|{2015-12-01 00:50:00, 2015-12-01 01:00:00}|2    |{2015-12-01 00:20:00, 2015-12-01 00:30:00}|2         |no         |
|citigroup|{2015-12-01 01:00:00, 2015-12-01 01:10:00}|1    |{2015-12-01 00:50:00, 2015-12-01 01:00:00}|2         |no         |
|citigroup|{2015-12-01 01:40:00, 2015-12-01 01:50:00}|1    |{2015-12-01 01:00:00, 2015-12-01 01:10:00}|1       

In [ ]:
citigroupEventCountEach10mDF_doubled = citigroupEventCountEach10mDF_check_doubled.filter(col("has_doubled") == "yes")
citigroupEventCountEach10mDF_doubled.show(truncate=False)

+---------+------------------------------------------+-----+------------------------------------------+----------+-----------+
|state    |window                                    |count|prev_window                               |prev_count|has_doubled|
+---------+------------------------------------------+-----+------------------------------------------+----------+-----------+
|citigroup|{2015-12-01 08:50:00, 2015-12-01 09:00:00}|12   |{2015-12-01 08:40:00, 2015-12-01 08:50:00}|3         |yes        |
|citigroup|{2015-12-01 14:00:00, 2015-12-01 14:10:00}|10   |{2015-12-01 13:50:00, 2015-12-01 14:00:00}|3         |yes        |
+---------+------------------------------------------+-----+------------------------------------------+----------+-----------+



In [ ]:
# Define a function to format and print the results
def print_citigroup_doubled_count(rows):
    for row in rows:
        previous_count = row['prev_count']
        current_count = row['count']
        start_time = row['window']['start'].strftime("%Y-%m-%d %H:%M:%S")
        end_time = row['window']['end'].strftime("%Y-%m-%d %H:%M:%S")
        print(f"The number of arrivals to Citigroup has doubled from {previous_count} to {current_count} at {start_time} - {end_time}")

# Print the results in the specified format
citigroup_doubled_rows = citigroupEventCountEach10mDF_doubled.select("prev_count", "count", "window").collect()  # Collect the rows to the driver
print_citigroup_doubled_count(citigroup_doubled_rows)

The number of arrivals to Citigroup has doubled from 3 to 12 at 2015-12-01 08:50:00 - 2015-12-01 09:00:00
The number of arrivals to Citigroup has doubled from 3 to 10 at 2015-12-01 14:00:00 - 2015-12-01 14:10:00
